<a href="https://colab.research.google.com/github/jcmachicao/MachineLearningAvanzado_UC_2024/blob/main/U3__aprendizaje_conjunto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo de aprendizaje conjunto

In [ ]:
!pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Datos de ejemplo
class TextDataset(Dataset):
    def __init__(self):

        # Textos codificados (usando embeddings ficticios para simplificar)
        self.data = torch.tensor([
            [1, 0, 0, 0, 0],  # "E=mc² es la base de la física moderna" -> Ciencia, Formal
            [0, 1, 0, 0, 0],  # "La gravedad es bien testaruda cuando actúa" -> Ciencia, Informal
            [0, 0, 1, 0, 0],  # "Los museos son escenciales para la cultura" -> Arte, Formal
            [0, 0, 0, 1, 0],  # "Qué locura fue el partido de tenis de ayer" -> Deportes, Informal
            [0, 0, 0, 0, 1],  # "Este pintor tiene obras loquísimas" -> Arte, Informal
        ], dtype=torch.float32)

        self.labels_tema = torch.tensor([0, 0, 1, 2, 1])  # Ciencia=0, Arte=1, Deportes=2
        self.labels_estilo = torch.tensor([0, 1, 0, 1, 1])  # Formal=0, Informal=1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels_tema[idx], self.labels_estilo[idx]

In [ ]:
# Modelo multitarea
class MultiTaskModel(nn.Module):
    def __init__(self, input_size, tema_classes, estilo_classes):
        super(MultiTaskModel, self).__init__()
        # Capas compartidas
        self.shared = nn.Linear(input_size, 16)
        self.relu = nn.ReLU()

        # Rama para tema
        self.tema_branch = nn.Linear(16, tema_classes)

        # Rama para estilo
        self.estilo_branch = nn.Linear(16, estilo_classes)

    def forward(self, x):
        # Pasar por la parte compartida
        shared_output = self.relu(self.shared(x))

        # Salidas de las ramas específicas
        tema_output = self.tema_branch(shared_output)
        estilo_output = self.estilo_branch(shared_output)

        return tema_output, estilo_output

In [ ]:
# Inicialización del modelo, datos y entrenamiento
dataset = TextDataset()
print(dataset)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

input_size = 5  # Tamaño del embedding ficticio
tema_classes = 3  # Ciencia, Arte, Deportes
estilo_classes = 2  # Formal, Informal

model = MultiTaskModel(input_size, tema_classes, estilo_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Entrenamiento
epochs = 20
for epoch in range(epochs):
    total_loss = 0.0
    for inputs, tema_labels, estilo_labels in dataloader:
        optimizer.zero_grad()

        # Forward pass
        tema_output, estilo_output = model(inputs)

        # Cálculo de las pérdidas
        loss_tema = criterion(tema_output, tema_labels)
        loss_estilo = criterion(estilo_output, estilo_labels)
        total_loss_batch = loss_tema + loss_estilo

        # Backpropagation y optimización
        total_loss_batch.backward()
        optimizer.step()

        total_loss += total_loss_batch.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

Epoch 1/20, Loss: 9.1526
Epoch 2/20, Loss: 8.5489
Epoch 3/20, Loss: 8.0630
Epoch 4/20, Loss: 7.6251
Epoch 5/20, Loss: 7.2268
Epoch 6/20, Loss: 6.7408
Epoch 7/20, Loss: 6.3148
Epoch 8/20, Loss: 5.8031
Epoch 9/20, Loss: 5.3055
Epoch 10/20, Loss: 4.7657
Epoch 11/20, Loss: 4.2077
Epoch 12/20, Loss: 3.6940
Epoch 13/20, Loss: 3.2271
Epoch 14/20, Loss: 2.7710
Epoch 15/20, Loss: 2.3482
Epoch 16/20, Loss: 2.0301
Epoch 17/20, Loss: 1.7207
Epoch 18/20, Loss: 1.4657
Epoch 19/20, Loss: 1.2413
Epoch 20/20, Loss: 1.0519


In [ ]:
# Prueba del modelo
test_input = torch.tensor([[1, 0, 0, 0, 0]], dtype=torch.float32)  # "E=mc²..."
tema_pred, estilo_pred = model(test_input)
print(f"Tema predicho: {torch.argmax(tema_pred, dim=1).item()}")
print(f"Estilo predicho: {torch.argmax(estilo_pred, dim=1).item()}")

Tema predicho: 0
Estilo predicho: 0
